# Feature engineering

This notebook takes care of the feature engineering part, in particular we're gonna add 3 new features to the dataset.

In [2]:
import pandas as pd
from os import path

RACES_PATH=path.join("..","dataset","races_cleaned.csv")
races_df=pd.read_csv(RACES_PATH)

CYCLISTS_PATH=path.join("..","dataset","cyclists_cleaned.csv")
cyclist_df=pd.read_csv(CYCLISTS_PATH)

The features we came up are:
- climbing_efficiency: we want to get a ratio between the climb total and the length, this  can help us unnderstand better theperformances for cyclists on different kind of slopeness on average for the race.
$$
climbing\_ efficiency= \frac{climb\_ total(m)}{length(KM)\times 1000}
$$
Note: length is multiplied by 1000 to get the same unit as the length
- competitive\_age: we calculate the difference the age a cyclist has at the time of the race, this is useful because we might track the performances  of a single cyclist in multiple races across time and determine,if possible, if there are ages that are more performant that others or any kind of pattern across ages.
$$
competitive\_age=current\_ race \_ date - cyclist\_ age
$$

- difficulty: this servers to estimate the difficulty an athlete might have when facing certain stages, it is weight sum of mutliple factors
    - elevation score : it's basically the climbing efficiency
    - terrain score : it's the difficulty associated to a single terrain type , the estimation is done using the percentage of presence of the terrain and the length of the coruse it self influenced by the BMI and the age
    
$$
terrain \_ score = \sum_i\frac{(\# \text{of times terrain i appears in the race})}{(\# \text{of stages for the race})}*(BMI*Age*\text{terrain $i$ difficulty})
$$
$$
difficulty=climbing\_ efficiency*(terrain \_ score \text{for cyclist})
$$

- convenience\_score: a ratio between points and difficulty, basically how much we gain from participating on certain races. The points of this feature is to find if there is any cyclist that tries to maximize the points gained while partecipating to the least difficult races or a brave one that goes to partecipate in difficult races to the it's capabilities and so on with any kind of behavioral pattern we can deduct.
$$
convenience\_ score=\frac{points}{difficulty}
$$

Note: difficulty was estimated using the climbing effiency in this case, better estimates might be obtained combining different features or taking into account charaterstics of the cyclist itself e.g. if i weight more i might be experienced but i have disadvantage in mountainous courses and perform better in flat ones compared to an inexperiencced competitor that weights less.

In [22]:
races_df['climbing_efficiency']=races_df['climb_total']/(races_df['length']*1000)
races_df['date']=pd.to_datetime(races_df['date'])

years_to_sub=races_df.merge(cyclist_df,left_on="cyclist",right_on="_url",how='inner')['birth_year'].astype('int32')
races_df['competitive_age']=races_df['date'].dt.year-years_to_sub

races_df['convenience_score']=races_df['points']/races_df['climbing_efficiency']

look for hgih capacity or performance at high altitudes (lungs capacity maybe), in 50's and 60's trainng and eating changed so athtlets in more recents races shoukd perform better on high altitudes.